 Implementing an Image Classifier to Detect Critical Road Filled Cracks uing ResNet-50 Architecture and Imagenet weights.

In [1]:
import zipfile
import os

def safe_unzip(source_zip, target_dir):
    """
    Safely extracts zip file contents to a target directory.

    :param source_zip: Path to the source zip file.
    :param target_dir: Directory where files will be extracted.
    """
    # Ensure the target directory exists
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Initialize the ZipFile object
    with zipfile.ZipFile(source_zip, 'r') as zip_ref:
        # Get a list of all archived file names from the zip
        listOfFileNames = zip_ref.namelist()

        # Iterate over each file
        for fileName in listOfFileNames:
            # Check filename endswith png or jpg to extract only images
            if fileName.endswith('.png') or fileName.endswith('.jpg'):
                # Extract a single file from zip
                try:
                    zip_ref.extract(fileName, path=target_dir)
                except zipfile.BadZipFile:
                    print(f"Skipping bad file: {fileName}")
                except Exception as e:
                    print(f"Error extracting {fileName}: {e}")
            else:
                print(f"Skipped non-image file: {fileName}")

# Usage
source_zip = '/content/drive/MyDrive/Masters_Thesis/Image_Classification/datasets/filled_crack_v2.zip'  # Path to your zip file
target_dir = '/content/data/'   # Target directory for unzipping

safe_unzip(source_zip, target_dir)

Skipped non-image file: filled_crack_v2/
Skipped non-image file: filled_crack_v2/.ipynb_checkpoints/
Skipped non-image file: filled_crack_v2/critical/
Skipped non-image file: filled_crack_v2/critical/.ipynb_checkpoints/
Skipped non-image file: filled_crack_v2/non_critical/
Skipped non-image file: filled_crack_v2/non_critical/.ipynb_checkpoints/


In [2]:
!rm -rf /content/data/filled_crack_v2/critical/.ipynb_checkpoints/
!rm -rf /content/data/filled_crack_v2/non_critical/.ipynb_checkpoints/
!rm -rf /content/data/filled_crack_v2/.ipynb_checkpoints/

In [20]:
!pip show torch torchvision

Name: torch
Version: 2.2.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision
---
Name: torchvision
Version: 0.17.1+cu121
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, torch
Required-by: fastai


In [21]:
import torch

import torch.nn.functional as F
import torch.nn as nn

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.transforms import functional as TF
from torchvision.models import resnet50, ResNet50_Weights

from torch.utils.data import DataLoader, Subset

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.model_selection import train_test_split

from tqdm import tqdm

import os

import glob

In [22]:
import os
root_data_dir = '/content/data/filled_crack_v2/'
len(os.listdir(root_data_dir+'non_critical'))

12584

In [23]:
# Load your dataset
root_data_dir = '/content/data/filled_crack_v2/'
dataset = ImageFolder(root = root_data_dir)

# Extract labels for stratification
labels = [sample[1] for sample in dataset.samples]

# Perform stratified split
train_idx, val_idx = train_test_split(
    range(len(labels)),
    test_size=0.1,
    stratify=labels,
    random_state=42
)

# Define subsets for train and validation using indices
train_subset = Subset(dataset, train_idx)
val_subset = Subset(dataset, val_idx)

In [24]:
class ResizePad:
    def __init__(self, target_size):
        self.target_size = target_size

    def __call__(self, img):
        width, height = img.size
        aspect_ratio = width / height

        if width > height:
            new_width = self.target_size
            new_height = int(self.target_size / aspect_ratio)
        else:
            new_height = self.target_size
            new_width = int(self.target_size * aspect_ratio)

        img = TF.resize(img, (new_height, new_width))
        padding_left = (self.target_size - new_width) // 2
        padding_top = (self.target_size - new_height) // 2
        padding_right = self.target_size - new_width - padding_left
        padding_bottom = self.target_size - new_height - padding_top

        img = TF.pad(img, (padding_left, padding_top, padding_right, padding_bottom), fill=0)
        return img

In [25]:
transform_train = transforms.Compose([
    ResizePad(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Randomly change the brightness, contrast, and saturation
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),  # Random affine transformations: rotations, translations, scale
    transforms.RandomVerticalFlip(),  # Flips the image vertically with a probability of 0.5
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    ResizePad(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [26]:
class CustomDataset:
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# Apply specific transformations
train_dataset = CustomDataset(train_subset, transform = transform_train)
val_dataset = CustomDataset(val_subset, transform = transform_val)

# Data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [27]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

In [28]:
for param in model.layer4.parameters():
    param.requires_grad = True

In [29]:
num_ftrs = model.fc.in_features
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [30]:
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 1)  # Assuming binary classification
)

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [31]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

In [32]:
def binary_accuracy(outputs, labels):
    # Apply sigmoid to convert outputs to probabilities
    probs = torch.sigmoid(outputs)
    # Convert probabilities to binary predictions
    preds = probs > 0.5
    # Compare with true labels
    correct = (preds == labels).float()  # Convert boolean to float for division
    acc = correct.sum() / len(correct)
    return acc

In [33]:
criterion = FocalLoss()

# Optimizer
optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad], lr=0.001, weight_decay=0.01)

# Scheduler
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.3)

In [34]:
# Saving model checkpoints
model_save_dir = '/content/drive/MyDrive/Masters_Thesis/Image_Classification/saved_models/'
def save_checkpoint(state, filename= os.path.join(model_save_dir, "model_checkpoint.pth.tar")):
    torch.save(state, filename)

In [35]:
# Training function
n_epochs_stop = 8
epochs_no_improve = 0
def train_model(num_epochs=25):
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        progress_bar_train = tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}', leave=False)
        for images, labels in progress_bar_train:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float()
            optimizer.zero_grad()
            outputs = model(images)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels)
            acc = binary_accuracy(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            running_acc += acc.item() * images.size(0)
            progress_bar_train.set_postfix({'train_loss': f'{loss.item():.4f}', 'train_acc': f'{acc.item():.4f}'})

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_acc / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        progress_bar_val = tqdm(val_loader, desc=f'Validation Epoch {epoch+1}/{num_epochs}', leave=False)
        with torch.no_grad():
            for images, labels in progress_bar_val:
                images, labels = images.to(device), labels.to(device)
                labels = labels.float()
                outputs = model(images)
                outputs = outputs.squeeze(1)
                loss = criterion(outputs, labels)
                acc = binary_accuracy(outputs, labels)
                val_loss += loss.item() * images.size(0)
                val_acc += acc.item() * images.size(0)

        val_loss /= len(val_loader.dataset)
        val_acc /= len(val_loader.dataset)
        tqdm.write(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        scheduler.step(val_loss)

        # Save checkpoint
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            save_checkpoint({'epoch': epoch + 1,
                             'state_dict': model.state_dict(),
                             'optimizer': optimizer.state_dict(),
                             'loss': val_loss,
                             'accuracy': val_acc})
        else:
            epochs_no_improve += 1

        if epochs_no_improve == n_epochs_stop:
            tqdm.write("Early stopping triggered after " + str(epoch + 1) + " epochs.")
            break  # Break out of the loop

train_model()

Epoch 1/25, Loss: 0.0289, Accuracy: 0.8495


Epoch 1/25 - Train Loss: 0.0289, Train Acc: 0.8495, Val Loss: 0.0289, Val Acc: 0.8513


Epoch 2/25, Loss: 0.0288, Accuracy: 0.8514


Epoch 2/25 - Train Loss: 0.0288, Train Acc: 0.8514, Val Loss: 0.0284, Val Acc: 0.8513


Epoch 3/25, Loss: 0.0284, Accuracy: 0.8514


...

Epoch 11/25 - Train Loss: 0.0279, Train Acc: 0.8514, Val Loss: 0.0279, Val Acc: 0.8513


Epoch 12/25, Loss: 0.0279, Accuracy: 0.8514


Epoch 12/25 - Train Loss: 0.0279, Train Acc: 0.8514, Val Loss: 0.0279, Val Acc: 0.8513


Epoch 13/25, Loss: 0.0279, Accuracy: 0.8514


Epoch 13/25 - Train Loss: 0.0279, Train Acc: 0.8514, Val Loss: 0.0279, Val Acc: 0.8513
Early stopping triggered after 13 epochs.


fatal: not a git repository (or any of the parent directories): .git
